In [1]:
import sys
sys.path.insert(0, '/home/skvortsova-ev/bert_cls/bert_environment/lib/python3.7/site-packages/')

In [2]:
import os
import argparse

import numpy
import torch
from allennlp.data.vocabulary import Vocabulary
from allennlp.common.params import Params
from allennlp.training.trainer import Trainer
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.models.model import Model
from summarus.readers import act_reader

import warnings
warnings.filterwarnings('ignore')
from summarus.readers import act_reader
from allennlp.data.dataloader import DataLoader

In [5]:
def set_seed(seed):
    numpy.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [6]:
model_path, train_path, val_path, seed, vocabulary_path, config_path = '/home/skvortsova-ev/notebooks/summarus/pgn_papers/','train_abstractive.csv', 'test_abstractive.csv', 10, '/home/skvortsova-ev/notebooks/summarus/pgn_papers/vocabulary/','/home/skvortsova-ev/notebooks/summarus/tests/configs/papers_pgn.json'

In [7]:
pretrained_path=None

In [8]:
assert os.path.isdir(model_path), "Model directory does not exist"
set_seed(seed)

config_path = config_path or os.path.join(model_path, "config.json")
assert os.path.isfile(config_path), "Config file does not exist"
params = Params.from_file(config_path)

vocabulary_path = vocabulary_path or os.path.join(model_path, "vocabulary")
assert os.path.exists(vocabulary_path), "Vocabulary is not ready, do not forget to run preprocess.py first"
vocabulary = Vocabulary.from_files(vocabulary_path)

reader_params = params.duplicate().pop("reader", default=Params({}))
reader = DatasetReader.from_params(reader_params)
train_dataset = reader.read(train_path)
val_dataset = reader.read(val_path) if val_path else None

if not pretrained_path:
    model_params = params.pop("model")
    model = Model.from_params(model_params, vocab=vocabulary)
else:
    model = Model.load(params, pretrained_path)
print(model)
print("Trainable params count: ", sum(p.numel() for p in model.parameters() if p.requires_grad))

train_dataset.index_with(vocabulary)
train_loader = DataLoader(train_dataset, batch_size = 15, shuffle = False)
val_dataset.index_with(vocabulary)
val_loader = DataLoader(val_dataset, batch_size = 15, shuffle = False)

trainer = Trainer.from_params(params.pop('trainer'), model=model, serialization_dir=model_path,
                            data_loader=train_loader, validation_data_loader=val_loader, 
                              patience = 3)
trainer.train()

PointerGeneratorNetwork(
  (_source_embedder): BasicTextFieldEmbedder(
    (token_embedder_tokens): Embedding()
  )
  (_encoder): LstmSeq2SeqEncoder(
    (_module): LSTM(64, 128, batch_first=True, bidirectional=True)
  )
  (_target_embedder): Embedding()
  (_decoder_cell): LSTMCell(320, 256)
  (_hidden_projection_layer): Linear(in_features=512, out_features=64, bias=True)
  (_output_projection_layer): Linear(in_features=64, out_features=63860, bias=True)
  (_p_gen_layer): Linear(in_features=1088, out_features=1, bias=True)
  (_attention): BahdanauAttention(
    (_decoder_hidden_projection_layer): Linear(in_features=256, out_features=256, bias=False)
    (_encoder_outputs_projection_layer): Linear(in_features=256, out_features=256, bias=False)
    (_v): Linear(in_features=256, out_features=1, bias=False)
    (_coverage_projection_layer): Linear(in_features=1, out_features=256, bias=False)
  )
)
Trainable params count:  22333749


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'best_epoch': 3,
 'peak_cpu_memory_MB': 1996.304,
 'peak_gpu_0_memory_MB': 9901,
 'peak_gpu_1_memory_MB': 2,
 'training_duration': '1:13:31.344790',
 'training_start_epoch': 0,
 'training_epochs': 5,
 'epoch': 5,
 'training_coverage_loss': 0.8514687808399851,
 'training_p_gen': 0.8057389709343253,
 'training_loss': 3.848850403222945,
 'training_reg_loss': 0.0,
 'training_cpu_memory_MB': 1996.304,
 'training_gpu_0_memory_MB': 9901,
 'training_gpu_1_memory_MB': 2,
 'validation_coverage_loss': 0.8563178211082647,
 'validation_p_gen': 0.7980787037149283,
 'validation_loss': 6.121881720460491,
 'validation_reg_loss': 0.0,
 'best_validation_coverage_loss': 0.883503285455115,
 'best_validation_p_gen': 0.760794092700084,
 'best_validation_loss': 6.111411993886217,
 'best_validation_reg_loss': 0.0}

In [9]:
model

PointerGeneratorNetwork(
  (_source_embedder): BasicTextFieldEmbedder(
    (token_embedder_tokens): Embedding()
  )
  (_encoder): LstmSeq2SeqEncoder(
    (_module): LSTM(64, 128, batch_first=True, bidirectional=True)
  )
  (_target_embedder): Embedding()
  (_decoder_cell): LSTMCell(320, 256)
  (_hidden_projection_layer): Linear(in_features=512, out_features=64, bias=True)
  (_output_projection_layer): Linear(in_features=64, out_features=63860, bias=True)
  (_p_gen_layer): Linear(in_features=1088, out_features=1, bias=True)
  (_attention): BahdanauAttention(
    (_decoder_hidden_projection_layer): Linear(in_features=256, out_features=256, bias=False)
    (_encoder_outputs_projection_layer): Linear(in_features=256, out_features=256, bias=False)
    (_v): Linear(in_features=256, out_features=1, bias=False)
    (_coverage_projection_layer): Linear(in_features=1, out_features=256, bias=False)
  )
)

In [10]:
import os
import argparse
import re
from typing import Dict

from allennlp.common.params import Params
from allennlp.models.model import Model
from allennlp_models.seq2seq import Seq2SeqPredictor
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
import torch
import nltk
import razdel

from summarus import *
from summarus.util.metrics import print_metrics
from summarus.predictors.sentences_tagger_predictor import SentencesTaggerPredictor


In [11]:
def punct_detokenize(text):
    text = text.strip()
    punctuation = ",.!?:;%"
    closing_punctuation = ")]}"
    opening_punctuation = "([}"
    for ch in punctuation + closing_punctuation:
        text = text.replace(" " + ch, ch)
    for ch in opening_punctuation:
        text = text.replace(ch + " ", ch)
    res = [r'"\s[^"]+\s"', r"'\s[^']+\s'"]
    for r in res:
        for f in re.findall(r, text, re.U):
            text = text.replace(f, f[0] + f[2:-2] + f[-1])
    text = text.replace("' s", "'s").replace(" 's", "'s")
    text = text.strip()
    return text


def get_abs_batches(reader: SummarizationReader,
                    test_path: str,
                    batch_size: int,
                    lowercase: bool = True) -> Dict:
    batch = []
    for source, target in reader.parse_set(test_path):
        source = source.strip()
        target = target.strip()
        if lowercase:
            source = source.lower() if lowercase else source
            target = target.lower() if lowercase else target
        sample = {"source": source, "target": target}
        batch.append(sample)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if batch:
        yield batch


def get_ext_batches(reader: SummarizationSentencesTaggerReader,
                    test_path: str,
                    batch_size: int,
                    lowercase: bool = True) -> Dict:
    batch = []
    for _, summary, sentences, tags in reader.parse_set(test_path):
        if lowercase:
            sentences = [sentence.lower() for sentence in sentences]
            summary = summary.lower()
        batch.append({"source_sentences": sentences, "sentences_tags": tags, "target": summary})
        if len(batch) == batch_size:
            yield batch
            batch = []
    if batch:
        yield batch


def run_baseline(batch, baseline):
    sources = [b.get("source") for b in batch]
    targets = [b.get("target") for b in batch]
    hyps = []
    for source in sources:
        source_sentences = nltk.sent_tokenize(source)
        if baseline.startswith("lead"):
            skip = 0
            if "skip" in baseline:
                skip = int(baseline.split("skip")[-1])
                n = int(baseline.split("skip")[0].replace("lead", ""))
            else:
                n = int(baseline.replace("lead", ""))
            if len(source_sentences) == 1:
                skip = 0
            hyp = " ".join(source_sentences[skip:skip+n]).strip()
        else:
            assert False
        hyps.append(hyp)
    return targets, hyps


def run_abs_model(predictor, batch):
    outputs = predictor.predict_batch_json(batch)
    targets = [b.get("target") for b in batch]
    hyps = []
    for output in outputs:
        decoded_words = output["predicted_tokens"]
        hyp = " ".join(decoded_words).strip()
        hyps.append(hyp)
    return targets, hyps


def run_ext_model(predictor, batch, top_n=3, border=None):
    assert top_n is not None or border is not None
    outputs = predictor.predict_batch_json(batch)
    targets = [b.get("target") for b in batch]
    hyps = []
    for sample, output in zip(batch, outputs):
        sentences = sample["source_sentences"]
        proba = torch.sigmoid(torch.Tensor(output["predicted_tags"])).tolist()
        sorted_proba = list(sorted(enumerate(proba), key=lambda x: -x[1]))
        if top_n is not None and border is None:
            indices = sorted_proba[:top_n]
        elif border is not None:
            indices = [(i, p) for i, p in sorted_proba if p > border]
            if top_n is not None:
                indices = indices[:top_n]
            if not indices:
                indices = sorted_proba[:1]
        indices.sort()
        hyp = [sentences[i] for i, _ in indices]
        hyp = " ".join(hyp).strip()
        hyps.append(hyp)
    return targets, hyps


def postprocess(ref, hyp, is_subwords=True, is_multiple_ref=False, detokenize_after=False, tokenize_after=True):
    hyp = hyp if not is_subwords else "".join(hyp.split(" ")).replace("▁", " ")
    if is_multiple_ref:
        reference_sents = ref.split(" s_s ")
        decoded_sents = hyp.split("s_s")
        hyp = [w.replace("<", "&lt;").replace(">", "&gt;").strip() for w in decoded_sents]
        ref = [w.replace("<", "&lt;").replace(">", "&gt;").strip() for w in reference_sents]
        hyp = " ".join(hyp)
        ref = " ".join(ref)
    ref = ref.strip()
    hyp = hyp.strip()
    if detokenize_after:
        hyp = punct_detokenize(hyp)
        ref = punct_detokenize(ref)
    if tokenize_after:
        hyp = " ".join([token.text for token in razdel.tokenize(hyp)])
        #hyp = hyp.replace("@ @ UNKNOWN @ @", "@@UNKNOWN@@")
        ref = " ".join([token.text for token in razdel.tokenize(ref)])
    return ref, hyp


def evaluate(test_path, batch_size, metric, mode,
             max_count, report_every, is_multiple_ref=False,
             model_path=None, model_config_path=None,
             detokenize_after=False, tokenize_after=False, meteor_jar=None):
    config_path = model_config_path or os.path.join(model_path, "config.json")
    params = Params.from_file(config_path)
    reader_params = params["reader"].duplicate()
    device = 0 if torch.cuda.is_available() else -1
    is_subwords = "tokenizer" in reader_params and reader_params["tokenizer"]["type"] == "subword"
    reader = DatasetReader.from_params(reader_params)
    hyps = []
    refs = []

    if mode == "abs":
        model = Model.load(params, model_path, cuda_device=device)
        model.eval()
        predictor = Seq2SeqPredictor(model, reader)
        print(model)
        print("Trainable params count: ", sum(p.numel() for p in model.parameters() if p.requires_grad))
    elif mode == "ext":
        model = Model.load(params, model_path, cuda_device=device)
        model.eval()
        predictor = SentencesTaggerPredictor(model, reader)
        print(model)
        print("Trainable params count: ", sum(p.numel() for p in model.parameters() if p.requires_grad))

    iter_batches = get_ext_batches if mode == "ext" else get_abs_batches
    for batch in iter_batches(reader, test_path, batch_size):
        if mode == "abs":
            batch_refs, batch_hyps = run_abs_model(predictor, batch)
        elif mode == "ext":
            batch_refs, batch_hyps = run_ext_model(predictor, batch)
        else:
            batch_refs, batch_hyps = run_baseline(batch, mode)
        for ref, hyp in zip(batch_refs, batch_hyps):
            ref, hyp = postprocess(ref, hyp, is_subwords if mode != 'ext' else False, is_multiple_ref, detokenize_after, tokenize_after)
            refs.append(ref)
            hyps.append(hyp)
            if len(hyps) % report_every == 0:
                print_metrics(refs, hyps, metric, meteor_jar)
            if max_count and len(hyps) >= max_count:
                break
    print_metrics(refs, hyps, metric, meteor_jar)